# Parameter recovery with LBA model

An important test for any model is to make sure that parameters are identifiable in the context of the task we are simulating. Here, we'll define a task and simulate it using a simple linear ballistic accumulator (LBA) model. Then we'll use Bayesian sampling to estimate the posterior probability of each parameter.

## Define a task to run

For this example, we'll assume only one trial type. Each trial involves selecting between one of three choices. 

First, we'll generate some simulated data to fit. For each trial, we need to generate a simulated response time in seconds and a response (0, 1, or 2).

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import aesara
import aesara.tensor as aet
import pymc as pm
from psireact import lba, model

You are running the v4 development version of PyMC which currently still lacks key features. You probably want to use the stable v3 instead which you can either install via conda or find on the v3 GitHub branch: https://github.com/pymc-devs/pymc/tree/v3


In [30]:
lba_model = lba.LBA()
param = {'A': 4, 'b': 8, 'v': [1, 2, 2.5], 's': 1, 'tau': 0}
n = 200
test_type = np.ones(n, dtype='int32')
df = lba_model.gen(test_type, param)

In [31]:
data = np.hstack(
    [
        df['rt'].to_numpy(),
        df['response'].to_numpy(),
        df['test'].to_numpy(),
    ]
)

In [32]:
x = np.arange(15).reshape(3, 5)
y = np.arange(15)
y.reshape(3, int(y.size / 3))

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

## Define a probabilistic model of the data

The model specifies a likelihood function. This function defines the probability of observing some data, given a set of parameters. We need to specify prior distributions for each model parameter. Fitting to data will then allow us to estimate posterior distributions, that is, estimates of the parameters that have been updated based on observed data.

In [33]:
with pm.Model() as graph:
    s = 1
    tau = 0
    b = 8
    A = pm.Uniform('A', lower=0, upper=b)
    v = pm.Uniform('v', lower=0, upper=10, shape=3)
    param = {'A': A, 'b': b, 'v': v, 's': s, 'tau': tau}
    def logp(x):
        # rt, response, test = x.reshape(3, int(x.size / 3))
        rt = x[:n]
        response = x[n:n * 2]
        test = x[n * 2:]
        p = lba_model.tensor_pdf(rt, response, test, param)
        return model.log_prob(p)
    t_resp = pm.DensityDist('t_resp', logp=logp, observed=data)

## Estimate posterior distributions

This will be a relatively easy model to fit, as we're using the same model that generated the data. We'll test whether the $v_i$ parameters and the $A$ parameter can be accurately estimated based on our simulated data.

Posterior distributions are estimated based on a "chain" of samples. Multiple chains are run (using multiple processor cores, if available, to run in parallel) for the sample posterior distribution. This lets us see if different random samples come out similar to one another.

In [34]:
trace = pm.sample(1000, tune=500, model=graph)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
/Users/morton/anaconda3/envs/devreact2/lib/python3.9/site-packages/pymc/model.py:925: FutureWarning: `Model.initial_point` has been deprecated. Use `Model.recompute_initial_point(seed=None)`.
  warnings.warn(
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [A, v]
/Users/morton/anaconda3/envs/devreact2/lib/python3.9/site-packages/pymc/model.py:925: FutureWarning: `Model.initial_point` has been deprecated. Use `Model.recompute_initial_point(seed=None)`.
  warnings.warn(
You are running the v4 development version of PyMC which currently still lacks key features. You probably want to use the stable v3 instead which you can either install via conda or find on the v3 GitHub branch: https://github.com/pymc-devs/pymc/tree/v3
You are running the v4 development version of PyMC which currently still lacks key features. You probably want to use the stable v3 instead which you can either install via conda or find on the v3

/Users/morton/anaconda3/envs/devreact2/lib/python3.9/site-packages/pymc/step_methods/hmc/quadpotential.py:258: RuntimeWarning: divide by zero encountered in true_divide
  np.divide(1, self._stds, out=self._inv_stds)
/Users/morton/anaconda3/envs/devreact2/lib/python3.9/site-packages/pymc/step_methods/hmc/quadpotential.py:258: RuntimeWarning: divide by zero encountered in true_divide
  np.divide(1, self._stds, out=self._inv_stds)
/Users/morton/anaconda3/envs/devreact2/lib/python3.9/site-packages/pymc/step_methods/hmc/quadpotential.py:237: RuntimeWarning: invalid value encountered in multiply
  return np.multiply(self._var, x, out=out)
/Users/morton/anaconda3/envs/devreact2/lib/python3.9/site-packages/pymc/model.py:925: FutureWarning: `Model.initial_point` has been deprecated. Use `Model.recompute_initial_point(seed=None)`.
  warnings.warn(
/Users/morton/anaconda3/envs/devreact2/lib/python3.9/site-packages/pymc/step_methods/hmc/quadpotential.py:237: RuntimeWarning: invalid value encounter

RuntimeError: Chain 0 failed.

## Evaluate the sampling

The pyMC3 sampler will print diagnostic information to give you an idea of whether there may be problems with the sampling. In practice, for complex models and especially hierarchical models, sampling will often be imperfect by the measures pyMC3 calculates. Our sample model, however, is simple enough that sampling should go pretty well. Different chains are plotted in different colors; they should lie about on top of each other in the distributions on the left. You also should not see much autocorrelation in the time plots on the right.

In [ ]:
h = pm.traceplot(trace)

## Get estimates of model parameters

This can be done in different ways, for example by calculating the mean over all samples or finding the maximum a posteriori parameters. Here, we take the mean for simplicity. 

These parameter estimates should be relatively close to the ones used to generate the data. Note, however, that estimation will never be perfect, and estimation accuracy depends on the number of data points we're fitting. For example, estimating the speed of the slowest accumulator is harder because we have fewer examples of that response in our simulated dataset.

In [ ]:
fixed = {'s': 1, 'tau': 0, 'b': 8}
group_vars = ['A', 'v']
param, subj_param = model.post_param(trace, fixed, group_vars)
param

## Generate data from fitted model

Now that we have parameter estimates, we can generate new data to see what the fit looks like compared to the original (simulated) data. A simple way to assess the fit is by plotting the relative frequency of reaction time as a function of each response option.

In [ ]:
sim = lba_model.gen(test_type, param, nrep=1)

In [ ]:
import psireact.plot as prp
g = prp.plot_fit(df, sim)